# Import and Loads

In [1]:
from ultralytics import YOLO
import os 
import sys
sys.path.append("/mnt/RAID/projects/FjordVision")
from models.probability_tree import ProbabilityTree
import torch
from anytree.importer import JsonImporter
from preprocessing.preprocessing import load_ground_truth_mask_xyn, convert_polygon_to_mask, calculate_binary_mask_iou
from utils.metrics import calculate_hierarchical_precision_recall, calculate_weighted_f1_score, hierarchical_similarity
torch.cuda.empty_cache()

# Function to divide the data into chunks of size n
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Load the YOLO model weights
model = YOLO('yolov8n-seg.pt')

importer = JsonImporter()
with open('data/coco.json', 'r') as f:
    root = importer.read(f)

classes_file = '/mnt/RAID/datasets/coco/classes.txt'

cls_names = []
with open(classes_file, 'r') as file:
    cls_names = [line.strip() for line in file]

subcat_names, cat_names, root_names = [], [], []
for node in root.descendants:
    if node.rank == 'subcategory':
        subcat_names.append(node.name)
    elif node.rank == 'category':
        cat_names.append(node.name)
    elif node.rank == 'root':
        root_names.append(node.name)

taxonomies = [cls_names, subcat_names, cat_names, root_names]

In [6]:
root_names

[]

# Construct Probability Tree

In [2]:
# Usage example
ontology_path = 'data/coco.json'  # Update this path as necessary
prob_tree = ProbabilityTree(ontology_path)

# Training Loop

In [5]:
# Define the image folder path
image_folder_path = '/mnt/RAID/datasets/COCO/images/val2017/'
frames = os.listdir(image_folder_path)
image_files_full_path = [image_folder_path + f for f in frames]

# Define the label folder path
label_folder_path = '/mnt/RAID/datasets/COCO/labels/val2017/'

class_index = []
with open(classes_file, 'r') as file:
    for line_number, line in enumerate(file, start=1):
        class_name = line.strip()
        class_index.append(class_name)

Y = []
Yhat = []
confidences = []
batch_size = 50

# Loop through batches of images
for image_batch in chunks(image_files_full_path, batch_size):

    with torch.no_grad():
        predictions = model(image_batch, stream=True)

    # Loop through the files in the image folder
    for file_name, prediction in zip(image_batch, predictions):
        # Check if the file is an image file
        if file_name.endswith('.jpg') or file_name.endswith('.png'):
            # Construct the corresponding label file name
            shape = prediction.orig_img.shape[:2]
            base_file_name = file_name.split('/')[-1].replace('.jpg', '.txt')
            label_file_path = label_folder_path + base_file_name
            NOGT = False
            # check if predictions are empty
            if len(prediction.boxes.cls) == 0:
                continue

            try:
                GT = load_ground_truth_mask_xyn(label_file_path)
            except FileNotFoundError:
                NOGT = True
                
            visited = len(GT)*[None]

            for cls, mask, conf in zip(prediction.boxes.cls, prediction.masks.xyn, prediction.boxes.conf):
                confidences.append(conf.item())
                m = convert_polygon_to_mask(mask, shape)
                best_iou = 0

                if NOGT:
                    Y.append(None)
                    Yhat.append(int(cls.item()))
                    continue
                else:
                    # calculate iou and find the best mask
                    for idx, (gcls, gmsk) in enumerate(GT):
                        g = convert_polygon_to_mask(gmsk, shape)
                        iou = calculate_binary_mask_iou(m, g)

                        if iou > best_iou and iou > 0.5:
                            best_iou = iou
                            best_g = g
                            best_gcls = gcls
                            visited[idx] = True
                            best_idx = idx

                if best_idx is not None and best_iou > 0.5:
                    visited[best_idx] = True

                if best_g is None:
                    Y.append(None)
                    Yhat.append(int(cls.item()))
                else:
                    Y.append(best_gcls)
                    Yhat.append(int(cls.item()))

            for vis in visited:
                if vis is None:
                    Y.append(GT[idx][0])
                    Yhat.append(None)

    # After processing each batch, clear unused memory from CUDA
    torch.cuda.empty_cache()

# Calculate scores without reclassification

In [25]:
# Calculate weighted precision, recall, and F1
precision, recall = calculate_hierarchical_precision_recall(Y, Yhat, confidences, taxonomies, prob_tree, threshold=0)
weighted_f1_score = calculate_weighted_f1_score(precision, recall)

In [26]:
precision

0.8116082735331355

In [27]:
recall

0.5414607358594179

In [28]:
weighted_f1_score

0.6495662970126433

# Update Predictions with uniform probability tree

In [68]:
# Calculate weighted precision, recall, and F1
precision, recall = calculate_hierarchical_precision_recall(Y, Yhat, confidences, taxonomies, prob_tree, threshold=0.3)
weighted_f1_score = calculate_weighted_f1_score(precision, recall)

# Scores with reclassifiation using uniform tree

In [69]:
precision

0.7882423271710082

In [70]:
recall

0.5176027232892776

In [71]:
weighted_f1_score

0.6248771628942967